In [2]:
# ----------------------
# Imports and Setup
# ----------------------

"""
Inflation-Aware Portfolio Allocation
=====================================
This module implements regime-based portfolio allocation strategies that adapt
to different inflation environments.

Integrated into Week 1 Project
Author: Enhanced from Jupyter Notebook
Date: 2026-01-27
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Optional, Tuple, Callable
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [4]:
# ----------------------
# Inflation Allocator
# ----------------------

class InflationRegimeAllocator:
    """
    Portfolio allocator that adjusts asset allocation based on inflation regimes.
    Adapted to work with S&P 500 returns from Week 1 project.
    """
    
    def __init__(
        self,
        df: pd.DataFrame,
        figures_dir: Optional[Path] = None
    ):
        """
        Initialize the allocator with returns and inflation data.
        
        Parameters
        ----------
        df : pd.DataFrame
            DataFrame containing real returns and inflation rates
        figures_dir : Path, optional
            Directory to save figures
        """
        self.df = df.copy()
        self.regime_stats = None
        self.allocations = None
        
        if figures_dir is None:
            self.figures_dir = Path.cwd() / "figures"
        else:
            self.figures_dir = Path(figures_dir)

        self.figures_dir.mkdir(parents=True, exist_ok=True)
    
    @staticmethod
    def default_regime_classifier(inflation_rate: float) -> str:
        """
        Default inflation regime classification.
        
        Parameters
        ----------
        inflation_rate : float
            Year-over-year inflation rate
            
        Returns
        -------
        str
            Regime classification ('Low', 'Moderate', 'High')
        """
        if inflation_rate < 0.01:
            return "Low (<1%)"
        elif inflation_rate < 0.03:
            return "Moderate (1-3%)"
        else:
            return "High (>3%)"
    
    def define_regimes(
        self,
        inflation_column: str = 'Inflation_Rate',
        classifier: Optional[Callable[[float], str]] = None
    ) -> pd.Series:
        """
        Classify each period into an inflation regime.
        
        Parameters
        ----------
        inflation_column : str, default='Inflation_Rate'
            Name of the inflation column to use
        classifier : Callable, optional
            Custom classification function
            
        Returns
        -------
        pd.Series
            Series containing regime classifications
        """
        if 'Inflation_Regime' in self.df.columns:
            return self.df['Inflation_Regime']
        
        if classifier is None:
            classifier = self.default_regime_classifier
        
        self.df['Inflation_Regime'] = (
            self.df[inflation_column].apply(classifier)
        )
        
        return self.df['Inflation_Regime']
    
    def calculate_regime_statistics(
        self,
        return_columns: Optional[List[str]] = None
    ) -> pd.DataFrame:
        """
        Calculate mean and standard deviation of returns by regime.
        
        Parameters
        ----------
        return_columns : List[str], optional
            List of return columns to analyze
            
        Returns
        -------
        pd.DataFrame
            Multi-index DataFrame with statistics by regime
        """
        if return_columns is None:
            return_columns = ['Nominal_Return', 'Real_Return']
        
        available_cols = [col for col in return_columns if col in self.df.columns]
        
        self.regime_stats = (
            self.df.groupby('Inflation_Regime')[available_cols]
            .agg(['mean', 'std', 'count'])
        )
        
        return self.regime_stats
    
    def define_regime_allocations(
        self,
        allocations: Optional[Dict[str, Dict[str, float]]] = None
    ) -> Dict[str, Dict[str, float]]:
        """
        Define allocation strategies for each inflation regime.
        
        For this single-asset case, we adjust equity exposure and cash holdings.
        
        Parameters
        ----------
        allocations : Dict[str, Dict[str, float]], optional
            Custom allocation dictionary
            
        Returns
        -------
        Dict[str, Dict[str, float]]
            Allocation weights by regime and asset
        """
        if allocations is None:
            # Default allocations for single equity asset (S&P 500)
            # Vary equity exposure based on regime
            self.allocations = {
                "Low (<1%)": {
                    "Equity": 0.90,  # High equity exposure in low inflation
                    "Cash": 0.10
                },
                "Moderate (1-3%)": {
                    "Equity": 0.70,  # Moderate exposure
                    "Cash": 0.30
                },
                "High (>3%)": {
                    "Equity": 0.50,  # Reduced exposure in high inflation
                    "Cash": 0.50
                }
            }
        else:
            self.allocations = allocations
        
        # Validate allocations sum to 1.0
        for regime, weights in self.allocations.items():
            total = sum(weights.values())
            if not np.isclose(total, 1.0, atol=0.01):
                raise ValueError(
                    f"Allocations for {regime} regime sum to {total:.2f}, not 1.0"
                )
        
        return self.allocations
    
    def calculate_portfolio_returns(
        self,
        return_col: str = 'Real_Return',
        cash_return: float = 0.0
    ) -> pd.Series:
        """
        Calculate portfolio returns using regime-based allocations.
        
        Parameters
        ----------
        return_col : str, default='Real_Return'
            Column name for equity returns
        cash_return : float, default=0.0
            Assumed return on cash holdings
            
        Returns
        -------
        pd.Series
            Time series of portfolio returns
        """
        if self.allocations is None:
            raise ValueError("Allocations not defined. Run define_regime_allocations first.")
        
        if 'Inflation_Regime' not in self.df.columns:
            raise ValueError("Regimes not defined. Run define_regimes first.")
        
        portfolio_returns = []
        
        for idx, row in self.df.iterrows():
            regime = row['Inflation_Regime']
            weights = self.allocations.get(regime, {'Equity': 1.0, 'Cash': 0.0})
            
            # Calculate weighted return
            equity_weight = weights.get('Equity', 0.0)
            cash_weight = weights.get('Cash', 0.0)
            
            period_return = (equity_weight * row[return_col] + 
                           cash_weight * cash_return)
            
            portfolio_returns.append(period_return)
        
        self.df['Portfolio_Return'] = portfolio_returns
        return pd.Series(portfolio_returns, index=self.df.index)
    
    def compare_strategies(
        self,
        benchmark_weight: float = 1.0,
        return_col: str = 'Real_Return',
        cash_return: float = 0.0,
        plot: bool = True
    ) -> pd.DataFrame:
        """
        Compare regime-based strategy against a static benchmark.
        
        Parameters
        ----------
        benchmark_weight : float, default=1.0
            Static equity allocation for benchmark (1.0 = 100% equity)
        return_col : str, default='Real_Return'
            Column name for returns
        cash_return : float, default=0.0
            Return on cash
        plot : bool, default=True
            Whether to create visualization
            
        Returns
        -------
        pd.DataFrame
            Comparison statistics
        """
        # Calculate benchmark returns (static allocation)
        self.df['Benchmark_Return'] = (
            benchmark_weight * self.df[return_col] + 
            (1 - benchmark_weight) * cash_return
        )
        
        # Calculate cumulative returns
        self.df['Portfolio_Cumulative'] = (1 + self.df['Portfolio_Return']).cumprod()
        self.df['Benchmark_Cumulative'] = (1 + self.df['Benchmark_Return']).cumprod()
        
        # Calculate statistics
        comparison = pd.DataFrame({
            'Strategy': ['Regime-Based', 'Benchmark (100% Equity)'],
            'Mean_Return': [
                self.df['Portfolio_Return'].mean(),
                self.df['Benchmark_Return'].mean()
            ],
            'Annualized_Return': [
                (1 + self.df['Portfolio_Return'].mean()) ** 12 - 1,
                (1 + self.df['Benchmark_Return'].mean()) ** 12 - 1
            ],
            'Std_Dev': [
                self.df['Portfolio_Return'].std(),
                self.df['Benchmark_Return'].std()
            ],
            'Annualized_Vol': [
                self.df['Portfolio_Return'].std() * np.sqrt(12),
                self.df['Benchmark_Return'].std() * np.sqrt(12)
            ],
            'Sharpe_Ratio': [
                (self.df['Portfolio_Return'].mean() / self.df['Portfolio_Return'].std()) * np.sqrt(12),
                (self.df['Benchmark_Return'].mean() / self.df['Benchmark_Return'].std()) * np.sqrt(12)
            ],
            'Cumulative_Return': [
                self.df['Portfolio_Cumulative'].iloc[-1] - 1,
                self.df['Benchmark_Cumulative'].iloc[-1] - 1
            ],
            'Max_Drawdown': [
                (self.df['Portfolio_Cumulative'] / self.df['Portfolio_Cumulative'].cummax() - 1).min(),
                (self.df['Benchmark_Cumulative'] / self.df['Benchmark_Cumulative'].cummax() - 1).min()
            ]
        })
        
        if plot:
            self._create_comparison_plots(comparison)
        
        return comparison
    
    def _create_comparison_plots(self, comparison: pd.DataFrame) -> None:
        """Create three visualization plots for the allocation analysis."""
        
        # Plot 1: Cumulative Returns Over Time
        fig, ax = plt.subplots(figsize=(14, 7))
        
        ax.plot(self.df.index, (self.df['Portfolio_Cumulative'] - 1) * 100,
                label='Regime-Based Strategy', linewidth=2.5, color='#2ecc71')
        ax.plot(self.df.index, (self.df['Benchmark_Cumulative'] - 1) * 100,
                label='100% Equity Benchmark', linewidth=2.5, color='#e74c3c', linestyle='--')
        
        ax.set_xlabel('Date', fontsize=12, fontweight='bold')
        ax.set_ylabel('Cumulative Return (%)', fontsize=12, fontweight='bold')
        ax.set_title('Inflation-Aware Allocation vs Buy-and-Hold Strategy',
                    fontsize=14, fontweight='bold', pad=15)
        ax.legend(fontsize=11, loc='best')
        ax.grid(True, alpha=0.3)
        ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
        
        plt.tight_layout()
        save_path = self.figures_dir / 'allocation_cumulative_returns.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {save_path}")
        plt.close()
        
        # Plot 2: Performance Metrics Comparison
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        metrics = [
            ('Annualized_Return', 'Annualized Return (%)', 100),
            ('Annualized_Vol', 'Annualized Volatility (%)', 100),
            ('Sharpe_Ratio', 'Sharpe Ratio', 1),
            ('Max_Drawdown', 'Maximum Drawdown (%)', 100)
        ]
        
        for idx, (metric, label, scale) in enumerate(metrics):
            ax = axes[idx // 2, idx % 2]
            values = comparison[metric].values * scale
            bars = ax.bar(comparison['Strategy'], values,
                         color=['#2ecc71', '#e74c3c'], alpha=0.7, edgecolor='black')
            
            # Add value labels
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.2f}',
                       ha='center', va='bottom', fontsize=10, fontweight='bold')
            
            ax.set_ylabel(label, fontsize=11, fontweight='bold')
            ax.set_title(label, fontsize=12, fontweight='bold')
            if metric != 'Max_Drawdown':
                ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
            ax.grid(True, alpha=0.3, axis='y')
            ax.tick_params(axis='x', rotation=15)
        
        plt.suptitle('Performance Metrics Comparison', fontsize=15, fontweight='bold', y=1.00)
        plt.tight_layout()
        
        save_path = self.figures_dir / 'allocation_performance_metrics.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {save_path}")
        plt.close()
        
        # Plot 3: Regime-Based Allocation Over Time
        fig, ax = plt.subplots(figsize=(14, 7))
        
        # Create allocation time series
        equity_allocation = []
        for idx, row in self.df.iterrows():
            regime = row['Inflation_Regime']
            equity_allocation.append(self.allocations[regime]['Equity'])
        
        # Plot as area chart
        ax.fill_between(self.df.index, 0, equity_allocation,
                       alpha=0.6, color='#3498db', label='Equity Allocation')
        ax.fill_between(self.df.index, equity_allocation, 1,
                       alpha=0.6, color='#95a5a6', label='Cash Allocation')
        
        # Add regime backgrounds
        regimes = self.df['Inflation_Regime'].values
        regime_colors = {'Low (<1%)': '#d4edda', 'Moderate (1-3%)': '#fff3cd', 'High (>3%)': '#f8d7da'}
        
        for i in range(len(self.df) - 1):
            if i == 0 or regimes[i] != regimes[i-1]:
                start_idx = self.df.index[i]
                # Find end of this regime
                end_idx = self.df.index[i]
                for j in range(i+1, len(self.df)):
                    if regimes[j] != regimes[i]:
                        end_idx = self.df.index[j-1]
                        break
                    end_idx = self.df.index[j]
                
                ax.axvspan(start_idx, end_idx, alpha=0.15,
                          color=regime_colors.get(regimes[i], 'white'))
        
        ax.set_xlabel('Date', fontsize=12, fontweight='bold')
        ax.set_ylabel('Portfolio Allocation', fontsize=12, fontweight='bold')
        ax.set_title('Dynamic Allocation Based on Inflation Regime',
                    fontsize=14, fontweight='bold', pad=15)
        ax.set_ylim(0, 1)
        ax.legend(fontsize=11, loc='upper left')
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add regime legend
        from matplotlib.patches import Patch
        regime_patches = [Patch(facecolor=color, alpha=0.15, label=regime)
                         for regime, color in regime_colors.items()]
        ax2 = ax.twinx()
        ax2.set_yticks([])
        ax2.legend(handles=regime_patches, title='Inflation Regime',
                  loc='upper right', fontsize=10)
        
        plt.tight_layout()
        
        save_path = self.figures_dir / 'allocation_regime_weights.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {save_path}")
        plt.close()
    
    def get_regime_distribution(self) -> pd.Series:
        """
        Get the distribution of time spent in each regime.
        
        Returns
        -------
        pd.Series
            Percentage of time in each regime
        """
        if 'Inflation_Regime' not in self.df.columns:
            raise ValueError("Regimes not defined. Run define_regimes first.")
        
        regime_counts = self.df['Inflation_Regime'].value_counts()
        regime_pcts = regime_counts / len(self.df) * 100
        
        return regime_pcts
    
    def run_full_analysis(
        self,
        return_col: str = 'Real_Return',
        benchmark_weight: float = 1.0,
        verbose: bool = True
    ) -> Dict:
        """
        Run complete inflation-aware allocation analysis.
        
        Parameters
        ----------
        return_col : str, default='Real_Return'
            Column name for returns
        benchmark_weight : float, default=1.0
            Benchmark equity allocation
        verbose : bool, default=True
            Whether to print results
            
        Returns
        -------
        Dict
            Dictionary containing all analysis results
        """
        results = {}
        
        if verbose:
            print("=" * 70)
            print("INFLATION-AWARE PORTFOLIO ALLOCATION ANALYSIS")
            print("=" * 70)
        
        # Define regimes
        self.define_regimes()
        regime_dist = self.get_regime_distribution()
        results['regime_distribution'] = regime_dist
        
        if verbose:
            print("\n1. Regime Distribution")
            print("-" * 70)
            for regime, pct in regime_dist.items():
                print(f"  {regime}: {pct:.1f}%")
        
        # Calculate regime statistics
        stats = self.calculate_regime_statistics()
        results['regime_statistics'] = stats
        
        if verbose:
            print("\n2. Asset Performance by Regime")
            print("-" * 70)
            print(stats.to_string())
        
        # Define allocations
        self.define_regime_allocations()
        results['allocations'] = self.allocations
        
        if verbose:
            print("\n3. Regime-Based Allocations")
            print("-" * 70)
            for regime, weights in self.allocations.items():
                print(f"\n  {regime}:")
                for asset, weight in weights.items():
                    print(f"    {asset}: {weight:.0%}")
        
        # Calculate portfolio returns
        self.calculate_portfolio_returns(return_col)
        
        # Compare with benchmark
        comparison = self.compare_strategies(benchmark_weight, return_col, plot=True)
        results['comparison'] = comparison
        
        if verbose:
            print("\n4. Strategy Comparison")
            print("-" * 70)
            print(comparison.to_string(index=False))
            
            print("\n" + "=" * 70)
            print("KEY INSIGHTS")
            print("=" * 70)
            
            regime_perf = comparison.iloc[0]
            benchmark_perf = comparison.iloc[1]
            
            if regime_perf['Sharpe_Ratio'] > benchmark_perf['Sharpe_Ratio']:
                print("\n• Regime-based strategy outperforms on risk-adjusted basis")
                print(f"  Sharpe improvement: {(regime_perf['Sharpe_Ratio'] - benchmark_perf['Sharpe_Ratio']):.3f}")
            
            if regime_perf['Annualized_Vol'] < benchmark_perf['Annualized_Vol']:
                vol_reduction = ((benchmark_perf['Annualized_Vol'] - regime_perf['Annualized_Vol']) / 
                               benchmark_perf['Annualized_Vol'] * 100)
                print(f"\n• Volatility reduced by {vol_reduction:.1f}%")
            
            if regime_perf['Max_Drawdown'] > benchmark_perf['Max_Drawdown']:
                print(f"\n• Maximum drawdown improved (less negative)")
                print(f"  Regime: {regime_perf['Max_Drawdown']:.1%} vs Benchmark: {benchmark_perf['Max_Drawdown']:.1%}")
        
        return results


def load_project_data(data_path: Optional[Path] = None) -> pd.DataFrame:
    """
    Load the combined analysis data from the Week 1 project.
    
    Parameters
    ----------
    data_path : Path, optional
        Path to combined_analysis.csv
        
    Returns
    -------
    pd.DataFrame
        Loaded and prepared dataframe
    """
    if data_path is None:
        project_root = Path.cwd()
        while project_root.name != "Inflation vs Market Returns Analysis":
            project_root = project_root.parent

        data_path = (
            project_root
            / "data"
            / "processed"
            / "combined_analysis.csv"
        )

    df = pd.read_csv(
        data_path,
        parse_dates=["Date"],
        index_col="Date"
    )

    return df


def main():
    """
    Run inflation-aware allocation analysis on Week 1 project data.
    """
    print("\nLoading Week 1 Project Data...")
    print("-" * 70)
    
    # Load data
    df = load_project_data()
    print(f"✓ Loaded {len(df)} observations")
    print(f"  Date range: {df.index.min()} to {df.index.max()}")
    
    # Initialize allocator
    allocator = InflationRegimeAllocator(df)
    
    # Run full analysis
    results = allocator.run_full_analysis(verbose=True)
    
    print("\n" + "=" * 70)
    print("✓ ANALYSIS COMPLETE")
    print("=" * 70)
    print(f"\nFigures saved to: {allocator.figures_dir}")
    
    return allocator, results


if __name__ == "__main__":
    allocator, results = main()


Loading Week 1 Project Data...
----------------------------------------------------------------------
✓ Loaded 130 observations
  Date range: 2012-05-31 00:00:00 to 2023-02-28 00:00:00
INFLATION-AWARE PORTFOLIO ALLOCATION ANALYSIS

1. Regime Distribution
----------------------------------------------------------------------
  Moderate (1-3%): 69.2%
  High (>3%): 16.9%
  Low (<1%): 13.8%

2. Asset Performance by Regime
----------------------------------------------------------------------
                 Nominal_Return                 Real_Return                
                           mean       std count        mean       std count
Inflation_Regime                                                           
High (>3%)            -0.000521  0.056257    22   -0.070203  0.060478    22
Low (<1%)              0.003925  0.038220    18    0.001147  0.039334    18
Moderate (1-3%)        0.012332  0.038524    90   -0.005746  0.039948    90

3. Regime-Based Allocations
---------------------